In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
from scipy.sparse import csr_matrix,csc_matrix

D:\developer\env\anaconda\envs\scBridge\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
H = sc.read_h5ad('pbmc.h5ad')

In [3]:
sc.pp.filter_cells(H,min_genes = 200)
sc.pp.filter_genes(H,min_cells=644)

In [4]:
sc.pp.filter_cells(R,min_genes = 200)
sc.pp.filter_genes(R,min_cells=644)

In [9]:
H.X

<78708x173026 sparse matrix of type '<class 'numpy.float32'>'
	with 54808255 stored elements in Compressed Sparse Row format>

In [12]:
# 读取RNA的基因表达矩阵
R = sc.read_10x_h5('GSM6284350_BMMC.RNA.hg19.gene.bc.matrices.h5')

In [14]:
R

AnnData object with n_obs × n_vars = 110678 × 51862
    var: 'gene_ids'

In [6]:
# 在R的obs中新增加一列
#R.obs.insert(R.obs.shape[1],'celltype','0')

In [57]:
meta_data = pd.read_excel('GSM6284346_BMMC.metadata.xlsx')

In [58]:
meta_data = pd.DataFrame(meta_data)
keep = ['atac.bc',' rna.bc',' celltype']
meta_data = meta_data.filter(keep,axis = 1)

In [59]:
meta_data

,atac.bc,rna.bc,celltype
0,"R1.001,R2.001,R3.112,P1.04","R1.001,R2.001,R3.112,P1.12",pro/pre-B
1,"R1.001,R2.002,R3.038,P1.04","R1.001,R2.002,R3.038,P1.12",Mono
2,"R1.001,R2.002,R3.157,P1.04","R1.001,R2.002,R3.157,P1.12",CD4
3,"R1.001,R2.003,R3.060,P1.04","R1.001,R2.003,R3.060,P1.12",pro/pre-B
4,"R1.001,R2.003,R3.061,P1.04","R1.001,R2.003,R3.061,P1.12",NK
...,...,...,...
78703,"R1.192,R2.191,R3.121,P1.04","R1.192,R2.191,R3.121,P1.12",CD4
78704,"R1.192,R2.191,R3.135,P1.04","R1.192,R2.191,R3.135,P1.12",NK
78705,"R1.192,R2.192,R3.001,P1.04","R1.192,R2.192,R3.001,P1.12",CD4
78706,"R1.192,R2.192,R3.033,P1.04","R1.192,R2.192,R3.033,P1.12",CD8


In [60]:
# 筛选没有标签的的RNA细胞,通过这一步骤的筛选之后RNA细胞barcode的数量变为78520,然后通过保留的barcode在后面的delet_rows_csr函数删除掉没有标签的细胞
RNA_raw_list = pd.DataFrame(R.obs.index.tolist(),columns=['Barcode'])
RNA_Barcode = []
bool = RNA_raw_list['Barcode'].isin(meta_data[' rna.bc'])
for i in range(0,len(R.obs.index.tolist())):
    if ~bool[i]:
        RNA_Barcode.append(i)
len(RNA_Barcode)
RNA_raw_list = RNA_raw_list.drop(RNA_Barcode,axis = 0)

In [62]:
# 删除metadata数据中没有标签的细胞
rename = {'atac.bc':'ATAC_Barcode',' rna.bc':'RNA_Barcode',' celltype':'Celltype'}
meta_data.rename(columns=rename,inplace=True)
bool_meta = meta_data['RNA_Barcode'].isin(RNA_raw_list['Barcode'])
bool_meta_index=[]
for i in range(0,len(bool_meta)):
    if ~bool_meta[i]:
        bool_meta_index.append(i)
meta_data = meta_data.drop(bool_meta_index,axis = 0)

In [63]:
meta_data

,ATAC_Barcode,RNA_Barcode,Celltype
0,"R1.001,R2.001,R3.112,P1.04","R1.001,R2.001,R3.112,P1.12",pro/pre-B
1,"R1.001,R2.002,R3.038,P1.04","R1.001,R2.002,R3.038,P1.12",Mono
2,"R1.001,R2.002,R3.157,P1.04","R1.001,R2.002,R3.157,P1.12",CD4
3,"R1.001,R2.003,R3.060,P1.04","R1.001,R2.003,R3.060,P1.12",pro/pre-B
4,"R1.001,R2.003,R3.061,P1.04","R1.001,R2.003,R3.061,P1.12",NK
...,...,...,...
78703,"R1.192,R2.191,R3.121,P1.04","R1.192,R2.191,R3.121,P1.12",CD4
78704,"R1.192,R2.191,R3.135,P1.04","R1.192,R2.191,R3.135,P1.12",NK
78705,"R1.192,R2.192,R3.001,P1.04","R1.192,R2.192,R3.001,P1.12",CD4
78706,"R1.192,R2.192,R3.033,P1.04","R1.192,R2.192,R3.033,P1.12",CD8


In [65]:
# 根据metadata中的barcode筛选ATAC数据于RNA共有的细胞
H_barcode = pd.DataFrame(H.obs.index.tolist(),columns=['Barcode'])
ATAC_Barcode = []
bool = H_barcode['Barcode'].isin(meta_data['ATAC_Barcode'])
for i in range(0,len(H.obs.index.tolist())):
    if ~bool[i]:
        ATAC_Barcode.append(i)
len(ATAC_Barcode)
new_ATAC_meta = H_barcode.drop(ATAC_Barcode,axis = 0)

,Barcode
0,"R1.001,R2.001,R3.112,P1.04"
1,"R1.001,R2.002,R3.038,P1.04"
2,"R1.001,R2.002,R3.157,P1.04"
3,"R1.001,R2.003,R3.060,P1.04"
4,"R1.001,R2.003,R3.061,P1.04"
...,...
78703,"R1.192,R2.191,R3.121,P1.04"
78704,"R1.192,R2.191,R3.135,P1.04"
78705,"R1.192,R2.192,R3.001,P1.04"
78706,"R1.192,R2.192,R3.033,P1.04"


In [73]:
#为dataframe重新建立索引
meta_data = meta_data.reset_index(drop=True)
RNA_raw_list = RNA_raw_list.reset_index(drop=True)
new_ATAC_meta = new_ATAC_meta.reset_index(drop=True)

In [160]:
# for i in range(0,78520):
#     for j in range(0,78520):
#         if meta_data[' rna.bc'][i] == RNA_raw_list['Barcode'][j]:
#             RNA_raw_list['celltype'][j] = meta_data[' celltype'][i]
#             meta_data = meta_data.drop(i)
#             break

In [81]:
# 合并meta_data与obs生成新的obs,目的是让Barcode于celltype对应上
new_RNA_obs = RNA_raw_list.merge(meta_data,left_on='Barcode', right_on='RNA_Barcode')
new_ATAC_obs = new_ATAC_meta.merge(meta_data, left_on='Barcode',right_on='ATAC_Barcode')

In [83]:
# 将obs中只保留CellType列
keep = ['Celltype']
new_RNA_obs = new_RNA_obs.filter(keep,axis = 1)
new_ATAC_obs = new_ATAC_obs.filter(keep,axis = 1)

In [70]:
# meta_data = meta_data.sort_values(by = ' rna.bc',ascending=True)
# RNA_raw_list= RNA_raw_list.sort_values(by = 'Barcode',ascending=True)

# 在稀疏矩阵X中剔除掉没有标签的细胞，生成新的matrix

In [68]:
def delete_rows_csr(mat, indices):
    """
    Remove the rows denoted by ``indices`` form the CSR sparse matrix ``mat``.
    """
    if not isinstance(mat, csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")
    mask = np.ones(mat.shape[0], dtype=bool)
    mask[indices] = False
    return mat[mask]
# 删除RNA中的细胞
mat_RNA = R.X
new_RNA_matrix = delete_rows_csr(mat_RNA,RNA_Barcode)
# 删除ATAC中的细胞
mat_ATAC = H.X
new_ATAC_matrix = delete_rows_csr(mat_ATAC,ATAC_Barcode)

# 通过新的obs和新的matrix生成新的matrix矩阵

# 生成最终的细胞数据

In [85]:
RNA_pro = anndata.AnnData(new_RNA_matrix,obs = new_RNA_obs)
ATAC_pro = anndata.AnnData(new_ATAC_matrix,obs = new_ATAC_obs)

D:\developer\env\anaconda\envs\scBridge\lib\site-packages\anndata\_core\anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
D:\developer\env\anaconda\envs\scBridge\lib\site-packages\anndata\_core\anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [86]:
RNA_pro.write('RNA_pro.h5ad')
ATAC_pro.write('ATAC_pro.h5ad')

# 以上步骤完成了细胞数量的筛选。下面的步骤是对基因进行筛选

In [87]:
data1 = sc.read_h5ad('RNA_pro.h5ad')
data2 = sc.read_h5ad('ATAC_pro.h5ad')

,Celltype
0,pro/pre-B
1,Mono
2,CD4
3,pro/pre-B
4,NK
...,...
78515,CD4
78516,NK
78517,CD4
78518,CD8
